#  5- Feature Selection
Selection based on voting: using many of the techniques (univariate and multivariate), we make a table with all the variables on the dataset and indicate the recommended variables for each technique, then we select a threshold for the total votings and on this basis we select the variables that will be used to train our models.

In [43]:
# Import libraries:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy import stats
from pyMechkar.analysis import Table1
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectFromModel



In [16]:
df = pd.read_csv("flat_file_after_feature_enrichment.csv")

def drop_additional_index_columns(_df):
    # dropping additional index columns that start with "Unnamed" - dropping these columns
    columns_to_drop = [x for x in _df.columns.to_list() if x.startswith("Unnamed")]
    print("dropping coulmns: ", columns_to_drop) # [Unamed..., Unamed..]
    return _df.drop(columns=columns_to_drop, axis=1, inplace=False)

df = drop_additional_index_columns(df)

C:\Users\itzikjan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (51) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


dropping coulmns:  ['Unnamed: 0']


In [17]:
df.head(5)

,listing_id,name,target_start_date_period,target_end_date_period,start_date_previous_period,end_date_previous_period,host_id,host_name,neighbourhood,latitude,...,host_total_listings_count_cat_host_total_listings_count_0%_to_25%,host_total_listings_count_cat_host_total_listings_count_50%_to_75%,host_total_listings_count_cat_host_total_listings_count_75%_to_100%,host_total_listings_count_cat_host_total_listings_count_missing,bedrooms_cat_bedrooms_0%_to_25%,bedrooms_cat_bedrooms_75%_to_100%,bedrooms_cat_bedrooms_missing,sqrt_bedrooms_cat_sqrt_bedrooms_0%_to_25%,sqrt_bedrooms_cat_sqrt_bedrooms_75%_to_100%,sqrt_bedrooms_cat_sqrt_bedrooms_missing
0,7071,BrightRoom with sunny greenview!,2019-06-01,2019-08-31,2018-11-07,2019-05-31,17391,Bright,Helmholtzplatz,52.543157,...,1,0,0,0,1,0,0,1,0,0
1,7071,BrightRoom with sunny greenview!,2019-07-01,2019-09-30,2018-11-07,2019-06-30,17391,Bright,Helmholtzplatz,52.543157,...,1,0,0,0,1,0,0,1,0,0
2,7071,BrightRoom with sunny greenview!,2019-08-01,2019-11-06,2018-11-07,2019-07-31,17391,Bright,Helmholtzplatz,52.543157,...,1,0,0,0,1,0,0,1,0,0
3,7071,BrightRoom with sunny greenview!,2019-02-01,2019-04-30,2018-11-07,2019-01-31,17391,Bright,Helmholtzplatz,52.543157,...,1,0,0,0,1,0,0,1,0,0
4,7071,BrightRoom with sunny greenview!,2019-03-01,2019-05-31,2018-11-07,2019-02-28,17391,Bright,Helmholtzplatz,52.543157,...,1,0,0,0,1,0,0,1,0,0


In [20]:
# Arrange the columns by type

def get_dict_of_df_types(pdf: pd.DataFrame) -> dict:
    """
    Arrange dataframe columns in the dictionary by types:
    For each type having a list of columns
    """
    d_of_columns_types = {}

    for c,t in zip(pdf.columns, pdf.dtypes):
        t_str = str(t)
        if d_of_columns_types.get(t_str) == None:
            d_of_columns_types[t_str] = [c]
        else:
            d_of_columns_types[t_str].append(c)
    return d_of_columns_types



In [23]:
d_of_columns_types = get_dict_of_df_types(df)
d_of_columns_types.keys()

dict_keys(['int64', 'object', 'float64'])

In [25]:
print("object columns:")
d_of_columns_types['object']

['name',
 'target_start_date_period',
 'target_end_date_period',
 'start_date_previous_period',
 'end_date_previous_period',
 'host_name',
 'neighbourhood',
 'last_review',
 'listing_url',
 'last_scraped',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'neighborhood_overview',
 'host_about',
 'host_since',
 'picture_url',
 'host_url',
 'host_location',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_verifications',
 'street',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'smart_location',
 'market',
 'country_code',
 'country',
 'is_location_exact',
 'amenities',
 'calendar_updated',
 'first_review',
 'calendar_last_scraped',
 'license',
 'concat_comments',
 'amenities_list',
 'host_verifications_list']

In [26]:
print("int64 columns:")
d_of_columns_types['int64']

int64 columns:


['listing_id',
 'host_id',
 'target_num_of_booked_days',
 'booked_up_target',
 'num_of_day_in_previous_period',
 'num_of_booked_days_in_previous_period',
 'kmean_cluster_availability',
 'target_num_of_day_in_period_minus_num_of_day_in_previous_period',
 'room_type_Private room_host_response_time_within an hour',
 'room_type_Private room_host_response_time_within a few hours',
 'room_type_Private room_host_response_time_missing_host_response_time',
 'room_type_Private room_host_response_time_within a day',
 'room_type_Private room_host_response_time_a few days or more',
 'room_type_Entire home/apt_host_response_time_within an hour',
 'room_type_Entire home/apt_host_response_time_within a few hours',
 'room_type_Entire home/apt_host_response_time_missing_host_response_time',
 'room_type_Entire home/apt_host_response_time_within a day',
 'room_type_Entire home/apt_host_response_time_a few days or more',
 'room_type_Shared room_host_response_time_within an hour',
 'room_type_Shared room_ho

In [27]:
print("float64 columns:")
d_of_columns_types['float64']

float64 columns:


['latitude',
 'longitude',
 'scrape_id',
 'zipcode',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'availability_365',
 'accommodates',
 'security_deposit',
 'cleaning_fee',
 'extra_people',
 'maximum_nights',
 'sqrt_calculated_host_listings_count',
 'sigmoid_calculated_host_listings_count',
 'sqrt_guests_included',
 'availability_60_minus_availability_90',
 'availability_60_div_availability_90',
 'availability_30_minus_availability_60',
 'availability_30_div_availability_60',
 'availability_365_minus_availability_90',
 'availability_365_div_availability_90',
 'availability_30_minus_availability_90',
 'availability_30_div_availability_90',
 'availability_365_minus_availability_60',
 'availability_365_div_availability_60',
 'target_num_of_day_in_period_div_num_of_day_in_previous_period',
 'occupancy_last_period_minus_availability_60',
 'occupancy_last_period_div_availability_60',
 'occupancy_last_period_minus_availability_365',
 'occupancy_last_period_div_availability_365',
 'occu

In [11]:
df['target_start_date_period'].unique()

array(['2019-06-01', '2019-07-01', '2019-08-01', '2019-02-01',
       '2019-03-01', '2019-04-01', '2019-05-01'], dtype=object)

In [12]:
df['target_end_date_period'].unique()

array(['2019-08-31', '2019-09-30', '2019-11-06', '2019-04-30',
       '2019-05-31', '2019-06-30', '2019-07-31', '2019-11-08'],
      dtype=object)

In [64]:
target_colums =  ['num_of_booked_days_in_previous_period', 'num_of_day_in_previous_period', 
                  'target_num_of_booked_days', 'target_num_of_day_in_period', 'booked_up_target']
columns_to_remove = target_colums + ['zipcode', 'availability_60_div_availability_90',
                                     'availability_30_div_availability_60','availability_365_div_availability_90',
                                     'availability_30_div_availability_90','availability_365_div_availability_60',
                                     'occupancy_last_period_div_availability_60',
                                     'occupancy_last_period_div_availability_365',
                                     'occupancy_last_period_div_availability_90']

x_columns = d_of_columns_types['int64'] + d_of_columns_types['float64'] 

x_columns = [col for col in x_columns if col not in columns_to_remove]
y_column = 'booked_up_target'

In [32]:
df[x_columns].head(4)

,listing_id,host_id,kmean_cluster_availability,target_num_of_day_in_period_minus_num_of_day_in_previous_period,room_type_Private room_host_response_time_within an hour,room_type_Private room_host_response_time_within a few hours,room_type_Private room_host_response_time_missing_host_response_time,room_type_Private room_host_response_time_within a day,room_type_Private room_host_response_time_a few days or more,room_type_Entire home/apt_host_response_time_within an hour,...,mean_of_max_temperatures_in_target_period,mean_precipitation_millimeters_in_previous_period,mean_precipitation_millimeters_in_target_period,mean_precipitation_days_in_previous_period,mean_precipitation_days_in_target_period,mean_sunshine_hours_in_day_in_previous_period,mean_sunshine_hours_in_day_in_target_period,mean_sunshine_hours_in_month_in_previous_period,mean_sunshine_hours_in_month_in_target_period,distance_from_center
0,7071,17391,1,-114,1,0,0,0,0,0,...,23.333333,43.571429,61.666667,9.285714,8.666667,3.500000,7.333333,106.428571,226.666667,4.551287
1,7071,17391,1,-144,1,0,0,0,0,0,...,22.333333,46.875000,53.333333,9.375000,8.000000,4.000000,6.500000,120.625000,205.000000,4.551287
2,7071,17391,1,-169,1,0,0,0,0,0,...,18.666667,47.777778,46.666667,9.222222,8.000000,4.388889,5.333333,133.888889,165.000000,4.551287
3,7071,17391,1,3,1,0,0,0,0,0,...,8.666667,46.666667,36.666667,10.333333,8.333333,1.666667,4.000000,48.333333,123.333333,4.551287


## Univariable Analysis

Each variable on the dataset is analyzed by comparing its relationship with the dependent variable.

### 1. Univariable Analysis with spearman correlation

In [72]:
# Spearman's rank correlation coefficient test, which is a nonparametric measure of rank correlation
# Checking for correlation between the variables and the target

def get_spearmanr_of_feature_with_target(_df, _x_columns, _y_column):
    rows_list = []
    for x_col in _x_columns:
        
        pearmanr_res  = stats.spearmanr(_df[x_col], _df[_y_column]) # (correlation, pvalue)
        row_dict = {"var": x_col, 
                    "spearmanr_correlation_with_target": pearmanr_res[0],
                    "spearmanr_pvalue_with_target": pearmanr_res[1] }   
        rows_list.append(row_dict)
    
    return pd.DataFrame(rows_list)

In [75]:
df_variables_selection = get_spearmanr_of_feature_with_target(df,x_columns, y_column)
df_variables_selection['spearmanr'] = np.where(df_variables_selection['spearmanr_pvalue_with_target'] < 0.05, 1, 0)
df_variables_selection = df_variables_selection[["var","spearmanr"]]

## Multivariable Analysis

Using the whole dataset and running predictive models that are able to return a list of recommended features by defining their influence in the model.

### 1. Multivariable Analysis with Lasso model

In [77]:
X = df[x_columns]
y = df[y_column]

In [79]:
lasso_model = Lasso(alpha=0.01).fit(X, y)
model = SelectFromModel(lasso_model, prefit=True)
model.get_support()

C:\Users\itzikjan\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3775.6095036594434, tolerance: 3.173229579257959
  positive)


array([False, False,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [80]:
df_variables_selection['lasso'] = model.get_support().astype('int64')

### 2. Multivariable Analysis with Random Forest model

In [66]:
rfmod = RandomForestClassifier().fit(X, y)
model = SelectFromModel(rfmod, prefit=True)
model.get_support()
df_variables_selection['lasso'] = model.get_support().astype('int64')

C:\Users\itzikjan\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array([ True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False,

In [ ]:
varSel['RandomForest'] = model.get_support().astype('int64')

### 3. Multivariable Analysis with Gradient Boosting model

In [68]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel

gbmod = GradientBoostingClassifier().fit(X, y)
model = SelectFromModel(gbmod, prefit=True)
model.get_support()

array([False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [ ]:
varSel['GradientBoost'] = model.get_support().astype('int64')
varSel

### 4. Multivariable Analysis with SVM model

In [69]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

svmmod = LinearSVC(C=0.01, penalty="l1",dual=False).fit(X, y)
model = SelectFromModel(svmmod, prefit=True)
model.get_support()

array([False, False,  True,  True,  True, False,  True, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [ ]:
varSel['SVM'] = model.get_support().astype('int64')
varSel

### 5. Multivariable Analysis with Ridge model

In [70]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import RidgeClassifier

ridge_model = RidgeClassifier(alpha=0.01).fit(X, y)
ridge_model = SelectFromModel(ridge_model, prefit=True)
ridge_model.get_support()

C:\Users\itzikjan\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=3.54797e-24): result may not be accurate.
  overwrite_a=True).T


array([False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False,  True,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False,  True,  True, False,  True, False, False, False, False,
       False, False, False, False,  True,  True, False,  True, False,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False,  True,
       False, False, False,  True, False,  True, False,  True,  True,
       False, False, False, False,  True,  True,  True, False, False,
       False,  True, False, False, False, False, False, False, False,
        True, False, False, False,  True,  True,  True,  True,  True,
       False,  True,

In [ ]:
stop here
tab1 = Table1(data=df[x_columns], y=y_column)

## Univariable Analysis with pyMechkar.analysis